# ANOVA mit Messwiederholungen

In dem letzten Teil haben wir uns ANOVAs angesehen, die sich mit einem Faktor ("Culmen Length (mm)") zwischen verschiedenen Individuuen beschäftigt. Also einem sogenannten between subjects factor. Die Variation zwischen den Gruppen also der between factor war der für uns interessante Effekt. Was ist aber, wenn wir wiederholt dieselbe Person messen, dann können wir nicht ohne weiteres dieselbe Logik anwenden, da wir die Variiation innerhalb der Gruppen, den within subjects factor, einfach als Fehler angesehen haben? Das wir das beim letzten Mal einfach so gemacht haben, verdeutlicht die Formel für den F-Wert.


$$ F = \frac{Signal}{Noise} = \frac{Explained\:variance}{Unexplained\:variance} = \frac{Between\:group\:variability}{Within\:group\:variability} $$

Unser Pinugindatensatz enthält allerdings keine Messwiederholungen. 

Daher kommt erstmal etwas Manipulation des Datensatzes, im Grunde füge ich eine id Spalte hinzu, sodass wir jeden individuellen Pinguin auseinanderhalten können. Außerdem gibt es einen Messzeitpunkt und wir simulieren ein leichtes Schnabelwachstum.

In [39]:
import pandas as pd

penguins = pd.read_csv("./penguins_classification.csv")

penguins.head()

,Culmen Length (mm),Culmen Depth (mm),Species
0,39.1,18.7,Adelie
1,39.5,17.4,Adelie
2,40.3,18.0,Adelie
3,36.7,19.3,Adelie
4,39.3,20.6,Adelie


In [2]:
penguins.shape

(342, 3)

In der nächsten Zelle passiert ziemlich viel Code, der nur dazu da ist unseren zweiten und dritten Zeitpunkt zu generieren und unseren Dataframe für eine repeated ANOVA vorzubereiten. Hierfür müssen wir verschiedene Dinge beachten. Eine repeated ANOVA braucht zum Beispiel einen Identifier, um zu wissen, welcher Wert zu welchem Individuum gehört (in unseren Simulierten Daten ändert sich die "Culmen Depth (mm)" nicht über die Zeit und die Species der Pinguine bleibt auch konstant, daher ist der Code für diese beiden parameter sehr ähnlich wie für die Identifier). Außerdem brauchen wir eine Variable (z.B. "time"), die den Messzeitpunkt kodiert und außerdem die Variable "Culmen Length (mm)", die sich über unsere drei Messzeitpunkte verändern soll. Dafür füge ich zu dem ursprünglichen Wert aus unseren Daten einen zufällig generierten Wert hinzu, um Schnabelwachstum zu simulieren. 

Ihr müsst nicht 100% verstehen, was in dieser Zelle passiert. Am allerwichtigsten sind die letzten zwei Zeile. Auch hier passiert ziemlich viel, wenn man sie Stück für Stück durchgeht wird es aber alles klar.

1. Wir entfernen die Spalten "Species" und "id" aus unserem neu erstellten DataFrame penguins_rm. 
2. Wir Gruppieren die Werte von "Culmen Length (mm)" und "Culmen Depth (mm)" nach der Zeit und lassen uns für beides den Mittelwert und die Standardabweichung ausgeben. Das ist quasi eine Weitere Methode um solche Übersichten wie mit `.describe` zu erstellen. Das könnte man auch statt `.agg(["mean", "std"])` anhängen. Die vielen Outputs von `.describe` sind mir hier allerdings zu unübersichtlich 

In [64]:
## generate t2 and t3

import numpy as np
# np.random.rand generiert einen zufälligen Vektor mit den in den Klammern angegebenen Dimensionen
penguins_rm = pd.DataFrame({
    "id": np.resize(np.arange(0, len(penguins)), 3*len(penguins)), # Id erstellen und dreimal wiederholen, da wir drei Messzeitpunkte haben
    "time": ["t1"] * len(penguins) + ["t2"] * len(penguins) + ["t3"] * len(penguins), # 
    "Culmen Length (mm)": pd.concat([
        penguins["Culmen Length (mm)"],
        penguins["Culmen Length (mm)"] + np.random.rand(len(penguins)),
        penguins["Culmen Length (mm)"] + 3 + np.random.rand(len(penguins))
        ]),
    "Culmen Depth (mm)": np.resize(penguins["Culmen Depth (mm)"], 3*len(penguins)),
    "Species": np.resize(penguins["Species"], 3*len(penguins))
})

penguins_rm_numerical = penguins_rm.drop(columns=["Species", "id"])
penguins_rm_numerical.groupby(["time"]).agg(["mean", "std"])

Culmen Length (mm)           Culmen Depth (mm)          
                   mean       std              mean       std
time                                                         
t1            43.921930  5.459584          17.15117  1.974793
t2            44.443177  5.471679          17.15117  1.974793
t3            47.393449  5.471752          17.15117  1.974793

In [22]:
import pingouin as pg

pg.rm_anova(
    data=penguins_rm,
    dv="Culmen Length (mm)",
    within="time",
    subject="id"
)

,Source,ddof1,ddof2,F,p-unc,np2,eps
0,time,1,341,981.518065,2.163734e-102,0.742159,1.0
